In [82]:
import pandas as pd
from collections import Counter
import numpy as np

In [88]:
df = pd.read_pickle("../tmp/product_textual.pickle")

In [94]:
from transformers import pipeline

summarizer = pipeline("summarization", model="Falconsai/text_summarization")

ARTICLE = list(df["pdt_product_detail_PRODUCTDESCRIPTION"])[0] + "The product includes: " + list(df["pdt_inclexcl_ENG_CONTENT"])[0]
print(summarizer(ARTICLE, max_length=1000, min_length=30, do_sample=False))

Your max_length is set to 1000, but your input_length is only 132. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)


[{'summary_text': 'La Cerdanya valley, in the Pyrenees, is one of the largest valleys in Europe . This route through Catalonia will be accompanied by the excellent Mediterranean climate . An excellent opportunity to get to know this region, its people, customs and this environment of great natural and cultural value .'}]


In [95]:
ARTICLE

'Our adventure starts off in\xa0La Cerdanya valley, in the Pyrenees, which is one of the largest valleys in Europe. Here we will cross part of Catalonia from\xa0La Cerdanya to\xa0Barcelona,\xa0travelling on secondary roads with beautiful landscapes.\n\nThis route through Catalonia will be accompanied by the excellent Mediterranean climate, which is ideal for cycling as it allows riders to enjoy the best sensations while on their bike.\n\nAn excellent opportunity to get to know this region, its people, customs and this environment of great natural and cultural value, which also offers great gastronomic options.'

In [96]:
df

,PRODUCTCODE,pdt_inclexcl_ENG_CONTENT,pdt_product_detail_PRODUCTDESCRIPTION
0,100123P1,nan,Our adventure starts off in La Cerdanya valley...
1,100123P2,nan,"In La Cerdanya, one of the largest valleys in ..."
2,100123P3,nan,We have planned this trip for those who love c...
3,100123P4,nan,"In La Cerdanya, one of the largest valleys in ..."
4,100123P5,nan,"In La Cerdanya, one of the largest valleys in ..."
...,...,...,...
21257,9973P33,nan,Get Ready to Sparkle in Paris: Your New Year'...
21258,9973P35,VIP Entrance to the Club,Get ready for the ultimate party experience i...
21259,9973P4,Skip the line. Visit 3/4 bars Clubs ( 4 Venues...,Riviera Bar Crawl Paris presents the ultimate ...
21260,9974P1,The Ice Bar Experience Admission Ticket . One ...,Experience the first ice bar at the beach in t...
